# Data Preprocessing

On this stage I do the following:

1. Utilizing [RecTools framework](https://github.com/MobileTeleSystems/RecTools) convert data to appropriate format
2. Using custom approach prepare data for it

## Prepare data for RecTools format

In [45]:
data_raw_dir = '../data/raw/'
data_interim_dir = '../data/interim/'
data_filenames = [f'u{t}.{split}' for t in ['1', '2', '3', '4', '5', 'a', 'b'] for split in ['base', 'test']]
data_filenames

['u1.base',
 'u1.test',
 'u2.base',
 'u2.test',
 'u3.base',
 'u3.test',
 'u4.base',
 'u4.test',
 'u5.base',
 'u5.test',
 'ua.base',
 'ua.test',
 'ub.base',
 'ub.test']

In [49]:
import pickle
import pandas as pd
from rectools import Columns
from rectools.dataset import Dataset


for filename in data_filenames:
    df = pd.read_csv(
        data_raw_dir + filename,
        sep='\t',
        names=[*Columns.Interactions],
        parse_dates=[Columns.Datetime]
    )
    df[Columns.Datetime] = pd.to_datetime(df[Columns.Datetime], unit='s')
    df[Columns.Weight] = df[Columns.Weight].astype(float)

    with open(data_interim_dir + filename + '.df.pickle', 'wb') as pickle_file:
        pickle.dump(df, pickle_file)

    dataset = Dataset.construct(df)

    with open(data_interim_dir + filename + '.pickle', 'wb') as pickle_file:
        pickle.dump(dataset, pickle_file)
    